In [12]:
################################################################################
#
####                               IMPORTING
#
################################################################################

import torch
import numpy as np
import torch.nn as nn
import pickle
import os
import scipy.io as sio
import random
from pathlib import Path

import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/model/remimo')
sys.path.append(os.path.dirname(os.getcwd()))

from model.remimo.sample_generator import sample_generator
from model.remimo.iterative_classifier import iterative_classifier
from model.remimo.utils_remimo import *

################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  General setup ###
######################
torch.manual_seed(123)
np.random.seed(123)

data_filename = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank.mat'
data_filename_test = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank2.mat'

mat_contents = sio.loadmat(data_filename)
mat_contents_test = sio.loadmat(data_filename_test)

useGPU = True # If true, and GPU is available, use it.
#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda'
else:
    device = 'cpu'

######################
### Parameter of the system and model ###
######################
NR = 64
NT_list = np.arange(16, 33)
# NT_list = np.arange(4,8)
NT_prob = NT_list/NT_list.sum()
mod_n = 16
d_transmitter_encoding = NR
d_model = 512
n_head = 8
nhid = d_model*4
nlayers = 16
dropout = 0.0

epoch_size = 5000
train_iter = 130*epoch_size

# Batch sizes for training and validation sets
train_batch_size = 256
mini_validtn_batch_size = 2500

learning_rate = 1e-4

corr_flag = True
batch_corr = True
rho_low = 0.55
rho_high = 0.75

validtn_NT_list = np.asarray([16, 32])
snrdb_list = {16:np.arange(11.0, 22.0), 32:np.arange(18.0, 25.0)}
factor_list = (validtn_NT_list/validtn_NT_list.sum())/snrdb_list[16].size

model_filename = './remimo_model_3gpp.pth'
curr_accr = './curr_accr.txt'
load_pretrained_model = False
save_interim_model = True
save_to_file = False

################################################################################
#
####                              MAIN
#
################################################################################


######################
### Functions ###
######################
def get_snr_range(NT):
    peak = NT*(5.0/16.0) + 6.0
    snr_low = peak
    snr_high = peak+10.0
    return (snr_low, snr_high)


def validate_model_given_data(model, validtn_H, validtn_y, validtn_j_indices, validtn_noise_sigma, device, criterion=None):
    with torch.no_grad():

        validtn_H = validtn_H.to(device=device).float()
        validtn_y = validtn_y.to(device=device).float()
        validtn_noise_sigma = validtn_noise_sigma.to(device=device).float()
        validtn_out = model.forward(validtn_H, validtn_y, validtn_noise_sigma)

        if (criterion):
            validtn_j_indices = validtn_j_indices.to(device=device)
            loss = loss_function(criterion, validtn_out, validtn_j_indices, nlayers)
            validtn_j_indices = validtn_j_indices.to(device='cpu')

        validtn_out = validtn_out[-1].to(device='cpu')
        accr = accuracy(validtn_out, validtn_j_indices)

        del validtn_H, validtn_y, validtn_noise_sigma, validtn_out, validtn_j_indices

        if (criterion):
            return accr, loss.item()
        else:
            return accr, None

def mini_validation(model, mini_validation_dict, i, device, criterion=None, save_to_file=True):
    result_dict = {int(NT):{} for NT in validtn_NT_list}
    loss_list = []
    for index,NT in enumerate(validtn_NT_list):
        for snr in snrdb_list[NT]:
            big_validtn_H, big_validtn_y, big_validtn_j_indices, big_noise_sigma = mini_validation_dict[NT][snr]
            accr, loss = validate_model_given_data(model, big_validtn_H, big_validtn_y, big_validtn_j_indices, big_noise_sigma, device, criterion)
            result_dict[NT][snr] = accr
            loss_list.append(loss*factor_list[index])

    print('Validtn result, Accr for 16 : ', result_dict[16])
    print('Validation resut, Accr for 32 : ', result_dict[32])
    if (save_to_file):
        with open(curr_accr, 'w') as f:
            print((i, result_dict), file=f)
        print('Saved intermediate validation results at : ', curr_accr)

    if (criterion):
        return np.sum(loss_list)

def generate_big_validtn_data(H, generator, batch_size, corr_flag, rho, batch_corr, rho_low, rho_high):
    validtn_data_dict = {int(NT):{} for NT in validtn_NT_list}
    for NT in validtn_NT_list:
        Haux1 = torch.tensor(H[random.sample(range(3276), mini_validtn_batch_size), :, :])#Pick up NT random users from 100.
        Haux = torch.tensor(Haux1[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
        Hr = torch.real(Haux)
        Hi = torch.imag(Haux)
        h1 = torch.cat((Hr, -1. * Hi), dim=2)
        h2 = torch.cat((Hi, Hr), dim=2)
        big_validtn_H = torch.cat((h1, h2), dim=1)

        for snr in snrdb_list[NT]:
            big_validtn_y, x, big_validtn_j_indices, big_noise_sigma = generator.give_batch_data_Hinput(big_validtn_H, int(NT), snr_db_min=snr, snr_db_max=snr, batch_size=batch_size)	
            validtn_data_dict[int(NT)][snr] = (big_validtn_H, big_validtn_y , big_validtn_j_indices, big_noise_sigma)
    return validtn_data_dict

def save_model_func(model, optimizer):
    torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, model_filename)
    print('******Model Saved********** at directory : ', model_filename)


def train(model, optimizer, lr_scheduler, generator , device='cpu'):
    H = mat_contents_test['H_bank']
    mini_validation_dict = generate_big_validtn_data(H, generator, mini_validtn_batch_size, corr_flag, None, batch_corr, rho_low, rho_high)
    # Fix loss criterion
    criterion = nn.CrossEntropyLoss().to(device=device)
    model.train()
    epoch_count = 1

    for i in range(1, train_iter+1):

        # Randomly select number of transmitters
        NT = np.random.choice(NT_list, p=NT_prob)
        rho = np.random.triangular(rho_low, rho_high, rho_high)

        snr_low, snr_high = get_snr_range(NT)
        
        H = mat_contents['H_bank']
        Haux1 = torch.tensor(H[random.sample(range(3276), train_batch_size), :, :])#Pick up NT random users from 100.
        Haux = torch.tensor(Haux1[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
        Hr = torch.real(Haux)
        Hi = torch.imag(Haux)
        h1 = torch.cat((Hr, -1. * Hi), dim=2)
        h2 = torch.cat((Hi, Hr), dim=2)
        H_train = torch.cat((h1, h2), dim=1)
        
        
        y, x, j_indices, noise_sigma = generator.give_batch_data_Hinput(H_train, NT, snr_db_min=snr_low,
                                                                snr_db_max=snr_high, 
                                                                batch_size = train_batch_size)
        H = H_train.to(device=device).float()
        y = y.to(device=device).float()
        noise_sigma = noise_sigma.to(device=device).float()

        out = model.forward(H,y, noise_sigma)

        del H, y, noise_sigma

        j_indices = j_indices.to(device=device)
        loss = loss_function(criterion, out, j_indices, nlayers)
        del j_indices, out
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_item = loss.item()
        del loss

        if (i%epoch_size==0):
            print('iteration number : ', i, 'Epoch : ', epoch_count, 'User : ', NT, 'loss : ', loss_item)
            print('Now validating')

            model.eval()
            mini_validtn_loss = mini_validation(model, mini_validation_dict, i, device, criterion, save_to_file)
            print('Mini validation loss : ', mini_validtn_loss)
            lr_scheduler.step(mini_validtn_loss)

            model.train()
            if (save_interim_model):
                save_model_func(model, optimizer)

            epoch_count = epoch_count+1

            

In [13]:
################################################################################
#
####                              MAIN RUN
#
################################################################################

generator = sample_generator(train_batch_size, mod_n, NR)
model = iterative_classifier(d_model, n_head, nhid, nlayers, mod_n, NR, d_transmitter_encoding, generator.real_QAM_const, generator.imag_QAM_const, generator.constellation, device, dropout)
model = model.to(device=device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

if (load_pretrained_model):
    checkpoint = torch.load(model_filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min', 0.91, 0, 0.0001, 'rel', 0, 0, 1e-08, verbose = True)
    print('*******Successfully loaded pre-trained model***********')
else:
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min', 0.91, 0, 0.0001, 'rel', 0, 0, 1e-08, verbose = True)

train(model, optimizer, lr_scheduler, generator, device)

/tmp/ipykernel_82322/2057404024.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Haux = torch.tensor(Haux1[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
/tmp/ipykernel_82322/2057404024.py:188: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Haux = torch.tensor(Haux1[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.


iteration number :  5000 Epoch :  1 User :  21 loss :  0.675198495388031
Now validating
Validtn result, Accr for 16 :  {11.0: tensor(0.7867), 12.0: tensor(0.8478), 13.0: tensor(0.8923), 14.0: tensor(0.9275), 15.0: tensor(0.9528), 16.0: tensor(0.9701), 17.0: tensor(0.9814), 18.0: tensor(0.9873), 19.0: tensor(0.9932), 20.0: tensor(0.9951), 21.0: tensor(0.9965)}
Validation resut, Accr for 32 :  {18.0: tensor(0.5432), 19.0: tensor(0.5607), 20.0: tensor(0.5786), 21.0: tensor(0.5877), 22.0: tensor(0.5973), 23.0: tensor(0.6100), 24.0: tensor(0.6177)}
Mini validation loss :  0.7122161505800304
******Model Saved********** at directory :  ./model_3gpp.pth
iteration number :  10000 Epoch :  2 User :  22 loss :  0.5933646559715271
Now validating
Validtn result, Accr for 16 :  {11.0: tensor(0.8128), 12.0: tensor(0.8739), 13.0: tensor(0.9171), 14.0: tensor(0.9487), 15.0: tensor(0.9700), 16.0: tensor(0.9819), 17.0: tensor(0.9914), 18.0: tensor(0.9945), 19.0: tensor(0.9965), 20.0: tensor(0.9982), 21.0

In [19]:
################################################################################
#
####                              MAIN RUN - TESTING
#
################################################################################


from collections import defaultdict
from pathlib import Path
import torch
import numpy as np
import torch.nn as nn
import pickle as pkl
import os
import scipy.io as sio
import random
from pathlib import Path

import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/model/remimo')
sys.path.append(os.path.dirname(os.getcwd()))

from model.remimo.sample_generator import sample_generator
from model.remimo.iterative_classifier import iterative_classifier
from model.remimo.utils_remimo import *

# Parameters
NR = 64
NT = 32
mod_n = 16
d_transmitter_encoding = NR
d_model = 512
n_head = 8
nhid = d_model*4
nlayers = 16
dropout = 0.0

# Batch sizes for training and validation sets
validtn_batch_size = 3276
validtn_iter = 500

M = int(np.sqrt(mod_n))
sigConst = np.linspace(-M+1, M-1, M) 
sigConst /= np.sqrt((sigConst ** 2).mean())
sigConst /= np.sqrt(2.) #Each complex transmitted signal will have two parts

validtn_NT_list = np.asarray([32, 32])
snrdb_list = {16:np.arange(11.0, 19.0), 32:np.arange(18.0, 25.0)}
save_result = False

dirMainPath = str(Path(os.getcwd()).parent.absolute())
data_filename = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank.mat'
data_filename_test = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank2.mat'

mat_contents = sio.loadmat(data_filename)
mat_contents_test = sio.loadmat(data_filename_test)

validtn_filename = './REMIMO_results'
model_filename = './remimo_model_3gpp.pth'


def accuracy(out, j_indices):
    out = out.permute(1,2,0)
    out = out.argmax(dim=1)
    accuracy = (out == j_indices).sum().to(dtype=torch.float32)
    return accuracy.item()/out.numel()

def bit_indices(indices, mod_n):
    real_indices = (indices//np.sqrt(mod_n)).to(dtype=torch.int32)
    imag_indices = (indices%np.sqrt(mod_n)).to(dtype=torch.int32)
    joint_bit_indices = torch.cat((real_indices, imag_indices), dim=-1)
    return joint_bit_indices

def sym_accuracy(out, j_indices):
    accuracy = (out == j_indices).sum().to(dtype=torch.float32)
    return accuracy.item()/out.numel()

def bit_accuracy(out, j_indices):
    out = out.permute(1,2,0)
    out = out.argmax(dim=1)
    bit_out_indices = bit_indices(out, mod_n)
    bit_j_indices = bit_indices(j_indices, mod_n)
    return sym_accuracy(bit_out_indices, bit_j_indices)

def validate_model_given_data(model, validtn_H, validtn_y, validtn_j_indices, validtn_noise_sigma, device):
    with torch.no_grad():

        validtn_H = validtn_H.to(device=device).float()
        validtn_y = validtn_y.to(device=device).float()
        validtn_noise_sigma = validtn_noise_sigma.to(device=device).float()
        validtn_out = model.forward(validtn_H, validtn_y, validtn_noise_sigma)

        validtn_out = validtn_out[-1].to(device='cpu')
        accr = accuracy(validtn_out, validtn_j_indices)

        del validtn_H, validtn_y, validtn_out, validtn_noise_sigma

    return accr


def validate_model(model, generator, device, save_result=True):
    H = mat_contents_test['H_bank']
    Haux1 = torch.tensor(H[random.sample(range(3276), validtn_batch_size), :, :])#Pick up NT random users from 100.
    Haux = torch.tensor(Haux1[:, :, random.sample(range(100), 32)])#Pick up NT random users from 100.

    Hr = torch.real(Haux)
    Hi = torch.imag(Haux)
    h1 = torch.cat((Hr, -1. * Hi), dim=2)
    h2 = torch.cat((Hi, Hr), dim=2)
    big_validtn_H = torch.cat((h1, h2), dim=1)

    result_dict = {int(NT): defaultdict(int) for NT in validtn_NT_list}
    for iter in range(validtn_iter):
        validtn_data_dict = generate_big_validtn_data(big_validtn_H, generator, validtn_batch_size)
        for NT in validtn_NT_list:
            for snr in snrdb_list[NT]:
                big_validtn_H, big_validtn_y, big_validtn_j_indices, big_noise_sigma = validtn_data_dict[NT][snr]
                accr = validate_model_given_data(model, big_validtn_H, big_validtn_y, big_validtn_j_indices, big_noise_sigma, device)
                result_dict[NT][snr] =  result_dict[NT][snr] + (accr-result_dict[NT][snr])/float(iter+1.0)

        if (save_result):
            with open(validtn_filename, 'wb') as handle:
                pkl.dump(result_dict, handle, protocol=pkl.HIGHEST_PROTOCOL)
            print('Intermediate Test results saved at : ', validtn_filename)
        print('Big Validtn result, Accr for 16 : ', result_dict[32])
        print('Big Validation resut, Accr for 32 : ', result_dict[32])


def generate_big_validtn_data(big_validtn_H, generator, batch_size):
    validtn_data_dict = {int(NT):{} for NT in validtn_NT_list}
    for NT in validtn_NT_list:

        for snr in snrdb_list[NT]:
            big_validtn_y, x, big_validtn_j_indices, big_noise_sigma = generator.give_batch_data_Hinput(big_validtn_H, int(NT), snr_db_min=snr, snr_db_max=snr, batch_size=batch_size)	
            validtn_data_dict[int(NT)][snr] = (big_validtn_H, big_validtn_y , big_validtn_j_indices, big_noise_sigma)
    return validtn_data_dict

def test(model, generator, device):
    model.eval()

    # Testing Trained Network
    validate_model(model, generator, device, save_result)

generator = sample_generator(validtn_batch_size, mod_n, NR)
device = 'cuda'
model = iterative_classifier(d_model, n_head, nhid, nlayers, mod_n, NR, d_transmitter_encoding, generator.real_QAM_const, generator.imag_QAM_const, generator.constellation, device, dropout)
model = model.to(device=device)

checkpoint = torch.load(model_filename)
model.load_state_dict(checkpoint['model_state_dict'])
print('*******Successfully loaded pre-trained model*********** from directory : ', model_filename)

test(model, generator, device)
print('******************************** Now Testing **********************************************')


*******Successfully loaded pre-trained model*********** from directory :  ./model_3gpp.pth
Big Validtn result, Accr for 16 :  defaultdict(<class 'int'>, {18.0: 0.8689331501831502, 19.0: 0.9278083028083028, 20.0: 0.9579708485958486, 21.0: 0.9762286324786325, 22.0: 0.9879903083028083, 23.0: 0.9942574786324786, 24.0: 0.9960412851037851})
Big Validation resut, Accr for 32 :  defaultdict(<class 'int'>, {18.0: 0.8689331501831502, 19.0: 0.9278083028083028, 20.0: 0.9579708485958486, 21.0: 0.9762286324786325, 22.0: 0.9879903083028083, 23.0: 0.9942574786324786, 24.0: 0.9960412851037851})
Big Validtn result, Accr for 16 :  defaultdict(<class 'int'>, {18.0: 0.8704641712454212, 19.0: 0.9225355807387057, 20.0: 0.9585503472222222, 21.0: 0.977723882020757, 22.0: 0.9884052579365079, 23.0: 0.9943934104090354, 24.0: 0.9965993208180708})
Big Validation resut, Accr for 32 :  defaultdict(<class 'int'>, {18.0: 0.8704641712454212, 19.0: 0.9225355807387057, 20.0: 0.9585503472222222, 21.0: 0.977723882020757, 22

KeyboardInterrupt: 